In [1]:
# importing libraries
import numpy as np
import pickle

In [2]:
# Shape of Tic Tac Toe Board
BOARD_ROWS = 3
BOARD_COLS = 3

In [3]:
# Vectorized function to convert to integer
def f(x):
    return int(x)
f2 = np.vectorize(f)

#### Tic Tac Toe Game State

In [4]:
# class State => with state and play management for training
class State:
    def __init__(self, p1, p2):
        # initial board state
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        # String Board standings of players 
        self.boardHash = None
        # starting with player p1
        self.playerSymbol = 1
    
    # get unique hash of current board state
    def getHash(self):
        # converting board standing to array
        self.board = np.array([f2(i) for i in list(self.board)])
        # converting array board standings to string as boardHash
        self.boardHash = str(self.board.reshape(BOARD_COLS*BOARD_ROWS))
        return self.boardHash
    
    # check for the winner
    def winner(self):
        # checking winner for row
        for i in range(BOARD_ROWS):
            # checking winner for p1 for row
            if sum(self.board[i, :]) == 3:
                self.isEnd = True
                return 1
            # checking winner for p2 for row
            if sum(self.board[i, :]) == -3:
                self.isEnd = True
                return -1
        # checking winner for col
        for i in range(BOARD_COLS):
            # checking winner for p1 for row
            if sum(self.board[:, i]) == 3:
                self.isEnd = True
                return 1
            # checking winner for p2 for row
            if sum(self.board[:, i]) == -3:
                self.isEnd = True
                return -1
            
        # checking winner for diagonals
        diag_sum1 = sum([self.board[i, i] for i in range(BOARD_COLS)])
        diag_sum2 = sum([self.board[i, BOARD_COLS-i-1] for i in range(BOARD_COLS)])
        # checking winner for p1 for diagonals
        if diag_sum1 == 3 or diag_sum2 == 3:
            self.isEnd = True
            return 1
        # checking winner for p2 for diagonals
        if diag_sum1 == -3 or diag_sum2 == -3:
            self.isEnd = True
            return -1
        
        # checking for tie
        # no available positions
        if len(self.availablePositions()) == 0:
            self.isEnd = True
            return 0
        # not end
        self.isEnd = False
        return None
    
    # check for empty positions in the board
    def availablePositions(self):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == 0:
                    positions.append((i, j))  # need to be tuple
        return positions
    
    # updating the board state as player makes a move with playerSymbol
    def updateState(self, position):
        self.board[position] = self.playerSymbol
        # switch to another player
        self.playerSymbol = -1 if self.playerSymbol == 1 else 1
    
    # Giving rewards only when game ends
    # result = 1 => win for p1
    # result = -1 => win for p2
    def giveReward(self):
        # getting the game result
        result = self.winner()        
        # giving rewards according to result
        if result == 1:
            # giving rewards when p1 wins
            self.p1.feedReward(1)
            self.p2.feedReward(0)
        elif result == -1:
            # giving rewards when p2 wins
            self.p1.feedReward(0)
            self.p2.feedReward(1)
        else:
            # giving rewards when the game is a tie
            # giving less rewards for tie
            self.p1.feedReward(0.001)
            self.p2.feedReward(0.001)
    
    # resetting the board for new game
    def reset(self):
        # initializing the board and other flags for new game
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.boardHash = None
        self.isEnd = False
        self.playerSymbol = 1
    
    # playing function for player p1 playing with human
    # game play
    def play(self):
        while not self.isEnd:
            # Player 1
            # first move is always made by player p1
            
            # getting available positions in the board
            positions = self.availablePositions()
            # getting action for player p1
            p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
            # taking action and upating board state
            self.updateState(p1_action)
            self.showBoard()
            
            # check board status if it is tie/win
            win = self.winner()
            if win is not None:
                if win == 1:
                    # p1 wins
                    print(self.p1.name, "wins!")
                else:
                    # game is a tie
                    print("tie!")
                self.reset()
                break

            else:
                # Player 2 => Human
                
                # getting available positions in the board
                positions = self.availablePositions()
                # getting action for player p1
                p2_action = self.p2.chooseAction(positions)
                # taking action and upating board state
                self.updateState(p2_action)
                self.showBoard()
                
                # check board status if it is tie/win
                win = self.winner()
                if win is not None:
                    if win == -1:
                        # human player wins
                        print(self.p2.name, "wins!")
                    else:
                        # game is a tie
                        print("tie!")
                    self.reset()
                    break

    # Displaying game board                
    def showBoard(self):
        # palyer symbols
        # p1: x  p2/human: o
        for i in range(0, BOARD_ROWS):
            print('-------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == -1:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('-------------')    

### Player for best policy -> Simple Monte Carlo Method with Bellman Equation and exploitation rate of 70% -> 0.3

In [11]:
class Player:
    def __init__(self, name, exp_rate=0.3):
        self.name = name # name of the player
        self.states = []  # record all positions taken
        self.exp_rate = exp_rate # exploitation rate -> default as 0.3 => 70%
        # define the gamma value (discount factor)
        self.decay_gamma = 0.9
        self.states_value = {}  # state -> value
    
    
    # get unique hash of current board state
    def getHash(self, board):
        # converting board standing to array
        board = np.array([f2(i) for i in list(board)])
         # converting array board standings to string as boardHash
        boardHash = str(board.reshape(BOARD_COLS*BOARD_ROWS))
        return boardHash
    
    # Generating next action of the player
    def chooseAction(self, positions, current_board, symbol):
        # getting next action randomly on the basis of exploitation value
        if np.random.uniform(0, 1) <= self.exp_rate:
            # This part is mainly for training purpose only
            # take random action
            idx = np.random.choice(len(positions))
            action = positions[idx]
        else:
            # This part is for game play
            # For gameplay the exploitation value is 0 
            # Here next action is selected according to the saved policy 
            value_max = -999
            for p in positions:
                next_board = current_board.copy()
                next_board[p] = symbol
                next_boardHash = self.getHash(next_board)
                # getting the next action value from the saved policy of the player during training
                value = 0 if self.states_value.get(next_boardHash) is None else self.states_value.get(next_boardHash)
                # print("value", value)
                if value >= value_max:
                    value_max = value
                    action = p
        return action
    
    # appending a hash state
    def addState(self, state):
        self.states.append(state)
    
    # updating state values with calculated rewards
    def feedReward(self, reward):
        for st in reversed(self.states):
            if self.states_value.get(st) is None:
                self.states_value[st] = 0
             # Reward Calculation => Simple Monte Carlo Method with Bellman Equation
            self.states_value[st] = (reward + self.decay_gamma*self.states_value[st])
            reward = self.states_value[st]
    
    # Resetting the states
    def reset(self):
        self.states = []
    
    # Saving the players game policies
    def savePolicy(self):
        fw = open('policy_' + str(self.name), 'wb')
        pickle.dump(self.states_value, fw)
        fw.close()
        
    # loding the saved policy
    def loadPolicy(self, file):
        fr = open(file,'rb')
        self.states_value = pickle.load(fr)
        fr.close()

In [12]:
class HumanPlayer:
    def __init__(self, name):
        self.name = name 
    
    def chooseAction(self, positions):
        while True:
            row = int(input("Input your action row:"))
            col = int(input("Input your action col:"))
            action = (row, col)
            if action in positions:
                return action
    
    # append a hash state
    def addState(self, state):
        pass
    
    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        pass
            
    def reset(self):
        pass

In [13]:
p1 = Player("computer", exp_rate=0)
p1.loadPolicy("policy_final_m1")

p2 = HumanPlayer("human")

st = State(p1, p2)
st.play()

-------------
| x |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
Input your action row:2
Input your action col:2
-------------
| x |   |   | 
-------------
|   |   |   | 
-------------
|   |   | o | 
-------------
-------------
| x |   |   | 
-------------
|   |   | x | 
-------------
|   |   | o | 
-------------
Input your action row:2
Input your action col:1
-------------
| x |   |   | 
-------------
|   |   | x | 
-------------
|   | o | o | 
-------------
-------------
| x |   |   | 
-------------
|   |   | x | 
-------------
| x | o | o | 
-------------
Input your action row:1
Input your action col:1
-------------
| x |   |   | 
-------------
|   | o | x | 
-------------
| x | o | o | 
-------------
-------------
| x | x |   | 
-------------
|   | o | x | 
-------------
| x | o | o | 
-------------
Input your action row:0
Input your action col:2
-------------
| x | x | o | 
-------------
|   | o | x | 
-------------
| x | o | o | 
-------------


In [14]:
st.play()

-------------
| x |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
Input your action row:1
Input your action col:1
-------------
| x |   |   | 
-------------
|   | o |   | 
-------------
|   |   |   | 
-------------
-------------
| x |   |   | 
-------------
|   | o | x | 
-------------
|   |   |   | 
-------------
Input your action row:2
Input your action col:1
-------------
| x |   |   | 
-------------
|   | o | x | 
-------------
|   | o |   | 
-------------
-------------
| x | x |   | 
-------------
|   | o | x | 
-------------
|   | o |   | 
-------------
Input your action row:0
Input your action col:2
-------------
| x | x | o | 
-------------
|   | o | x | 
-------------
|   | o |   | 
-------------
-------------
| x | x | o | 
-------------
|   | o | x | 
-------------
| x | o |   | 
-------------
Input your action row:1
Input your action col:0
-------------
| x | x | o | 
-------------
| o | o | x | 
-------------
| x | o |   | 
-------------


In [15]:
st.play()

-------------
| x |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
Input your action row:1
Input your action col:1
-------------
| x |   |   | 
-------------
|   | o |   | 
-------------
|   |   |   | 
-------------
-------------
| x |   |   | 
-------------
|   | o | x | 
-------------
|   |   |   | 
-------------
Input your action row:2
Input your action col:0
-------------
| x |   |   | 
-------------
|   | o | x | 
-------------
| o |   |   | 
-------------
-------------
| x |   |   | 
-------------
|   | o | x | 
-------------
| o | x |   | 
-------------
Input your action row:0
Input your action col:2
-------------
| x |   | o | 
-------------
|   | o | x | 
-------------
| o | x |   | 
-------------
human wins!
